In [1]:
from pynq import Overlay,allocate
import numpy as np
import os

In [2]:
class bp_predictor:
    def __init__(self):
        self.lstm_ppg_fwts = np.load('lstm_forward_wts_ppg.npy',allow_pickle=True)
        self.lstm_ppg_rwts = np.load('lstm_backward_wts_ppg.npy',allow_pickle=True)
        self.conv_ppg_wts = np.load('conv_ppg_wts.npy',allow_pickle=True)
        self.lstm_ecg_fwts = np.load('lstm_forward_wts_ecg.npy',allow_pickle=True)
        self.lstm_ecg_rwts = np.load('lstm_backward_wts_ecg.npy',allow_pickle=True)
        self.conv_ecg_wts = np.load('conv_ecg_wts.npy',allow_pickle=True)
        self.overlay = Overlay('bp_final.bit')
        self.conv_cell = self.overlay.conv_lstm_0
        self.conv_reg = self.conv_cell.register_map
        self.dense_cell = self.overlay.dense_0
        self.dense_reg = self.dense_cell.register_map
    
    def conv_mem(self):
        self.conv_ip = allocate(shape=(1024,),dtype=np.float32)
        self.k1_buff = allocate(shape=(48,),dtype=np.float32)
        self.bias1_buff = allocate(shape=(16,),dtype=np.float32)
        self.k2_buff = allocate(shape=(1536,),dtype=np.float32)
        self.bias2_buff = allocate(shape=(32,),dtype=np.float32)
        self.h1 = allocate(shape=(32,),dtype=np.float32)
        self.h2 = allocate(shape=(32,),dtype=np.float32)


        self.w_f_for = allocate(shape=(1024,),dtype=np.float32)
        self.u_f_for = allocate(shape=(1024,),dtype=np.float32)
        self.b_f_for = allocate(shape=(32,),dtype=np.float32)

        self.w_f_rev = allocate(shape=(1024,),dtype=np.float32)
        self.u_f_rev = allocate(shape=(1024,),dtype=np.float32)
        self.b_f_rev = allocate(shape=(32,),dtype=np.float32)

        self.w_i_for = allocate(shape=(1024,),dtype=np.float32)
        self.u_i_for = allocate(shape=(1024,),dtype=np.float32)
        self.b_i_for = allocate(shape=(32,),dtype=np.float32)

        self.w_i_rev = allocate(shape=(1024,),dtype=np.float32)
        self.u_i_rev = allocate(shape=(1024,),dtype=np.float32)
        self.b_i_rev = allocate(shape=(32,),dtype=np.float32)

        self.w_c_for = allocate(shape=(1024,),dtype=np.float32)
        self.u_c_for = allocate(shape=(1024,),dtype=np.float32)
        self.b_c_for = allocate(shape=(32,),dtype=np.float32)

        self.w_c_rev = allocate(shape=(1024,),dtype=np.float32)
        self.u_c_rev = allocate(shape=(1024,),dtype=np.float32)
        self.b_c_rev = allocate(shape=(32,),dtype=np.float32)

        self.w_o_for = allocate(shape=(1024,),dtype=np.float32)
        self.u_o_for = allocate(shape=(1024,),dtype=np.float32)
        self.b_o_for = allocate(shape=(32,),dtype=np.float32)

        self.w_o_rev = allocate(shape=(1024,),dtype=np.float32)
        self.u_o_rev = allocate(shape=(1024,),dtype=np.float32)
        self.b_o_rev = allocate(shape=(32,),dtype=np.float32)

        self.h_curr_for = allocate(shape=(32,),dtype=np.float32)
        self.h_curr_rev = allocate(shape=(32,),dtype=np.float32)
    
    def conv_free_mem(self):
        self.conv_ip.free_buffer()
        self.k1_buff.free_buffer()
        self.bias1_buff.free_buffer()
        self.k2_buff.free_buffer()
        self.bias2_buff.free_buffer()
        self.h1.free_buffer()
        self.h2.free_buffer()

        self.w_f_for.free_buffer()
        self.u_f_for.free_buffer()
        self.b_f_for.free_buffer()

        self.w_f_rev.free_buffer()
        self.u_f_rev.free_buffer()
        self.b_f_rev.free_buffer()

        self.w_i_for.free_buffer()
        self.u_i_for.free_buffer()
        self.b_i_for.free_buffer()

        self.w_i_rev.free_buffer()
        self.u_i_rev.free_buffer()
        self.b_i_rev.free_buffer()

        self.w_c_for.free_buffer()
        self.u_c_for.free_buffer()
        self.b_c_for.free_buffer()

        self.w_c_rev.free_buffer()
        self.u_c_rev.free_buffer()
        self.b_c_rev.free_buffer()

        self.w_o_for.free_buffer()
        self.u_o_for.free_buffer()
        self.b_o_for.free_buffer()

        self.w_o_rev.free_buffer()
        self.u_o_rev.free_buffer()
        self.b_o_rev.free_buffer()   

    def dense_mem(self):
        self.dense_ip_1 = allocate(shape=(64,),dtype=np.float32)
        self.dense_ip_2 = allocate(shape=(64,),dtype=np.float32)
        self.dense_op = allocate(shape=(2,),dtype=np.float32)

    def dense_free_mem(self):
        self.dense_ip_1.free_buffer() 
        self.dense_ip_2.free_buffer() 
        self.dense_op.free_buffer()

    def load_ppg(self):
        
        np.copyto(self.k1_buff,self.conv_ppg_wts[0])
        np.copyto(self.bias1_buff,self.conv_ppg_wts[1])
        np.copyto(self.k2_buff,self.conv_ppg_wts[2])
        np.copyto(self.bias2_buff,self.conv_ppg_wts[3])        

        np.copyto(self.w_i_for,self.lstm_ppg_fwts[0])
        np.copyto(self.w_f_for,self.lstm_ppg_fwts[1])
        np.copyto(self.w_c_for,self.lstm_ppg_fwts[2])
        np.copyto(self.w_o_for,self.lstm_ppg_fwts[3])

        np.copyto(self.u_i_for,self.lstm_ppg_fwts[4])
        np.copyto(self.u_f_for,self.lstm_ppg_fwts[5])
        np.copyto(self.u_c_for,self.lstm_ppg_fwts[6])
        np.copyto(self.u_o_for,self.lstm_ppg_fwts[7])

        np.copyto(self.b_i_for,self.lstm_ppg_fwts[8])
        np.copyto(self.b_f_for,self.lstm_ppg_fwts[9])
        np.copyto(self.b_c_for,self.lstm_ppg_fwts[10])
        np.copyto(self.b_o_for,self.lstm_ppg_fwts[11])

        np.copyto(self.w_i_rev,self.lstm_ppg_rwts[0])
        np.copyto(self.w_f_rev,self.lstm_ppg_rwts[1])
        np.copyto(self.w_c_rev,self.lstm_ppg_rwts[2])
        np.copyto(self.w_o_rev,self.lstm_ppg_rwts[3])

        np.copyto(self.u_i_rev,self.lstm_ppg_rwts[4])
        np.copyto(self.u_f_rev,self.lstm_ppg_rwts[5])
        np.copyto(self.u_c_rev,self.lstm_ppg_rwts[6])
        np.copyto(self.u_o_rev,self.lstm_ppg_rwts[7])

        np.copyto(self.b_i_rev,self.lstm_ppg_rwts[8])
        np.copyto(self.b_f_rev,self.lstm_ppg_rwts[9])
        np.copyto(self.b_c_rev,self.lstm_ppg_rwts[10])
        np.copyto(self.b_o_rev,self.lstm_ppg_rwts[11])

    def load_ecg(self):
        np.copyto(self.k1_buff,self.conv_ecg_wts[0])
        np.copyto(self.bias1_buff,self.conv_ecg_wts[1])
        np.copyto(self.k2_buff,self.conv_ecg_wts[2])
        np.copyto(self.bias2_buff,self.conv_ecg_wts[3])        

        np.copyto(self.w_i_for,self.lstm_ecg_fwts[0])
        np.copyto(self.w_f_for,self.lstm_ecg_fwts[1])
        np.copyto(self.w_c_for,self.lstm_ecg_fwts[2])
        np.copyto(self.w_o_for,self.lstm_ecg_fwts[3])

        np.copyto(self.u_i_for,self.lstm_ecg_fwts[4])
        np.copyto(self.u_f_for,self.lstm_ecg_fwts[5])
        np.copyto(self.u_c_for,self.lstm_ecg_fwts[6])
        np.copyto(self.u_o_for,self.lstm_ecg_fwts[7])

        np.copyto(self.b_i_for,self.lstm_ecg_fwts[8])
        np.copyto(self.b_f_for,self.lstm_ecg_fwts[9])
        np.copyto(self.b_c_for,self.lstm_ecg_fwts[10])
        np.copyto(self.b_o_for,self.lstm_ecg_fwts[11])

        np.copyto(self.w_i_rev,self.lstm_ecg_rwts[0])
        np.copyto(self.w_f_rev,self.lstm_ecg_rwts[1])
        np.copyto(self.w_c_rev,self.lstm_ecg_rwts[2])
        np.copyto(self.w_o_rev,self.lstm_ecg_rwts[3])

        np.copyto(self.u_i_rev,self.lstm_ecg_rwts[4])
        np.copyto(self.u_f_rev,self.lstm_ecg_rwts[5])
        np.copyto(self.u_c_rev,self.lstm_ecg_rwts[6])
        np.copyto(self.u_o_rev,self.lstm_ecg_rwts[7])

        np.copyto(self.b_i_rev,self.lstm_ecg_rwts[8])
        np.copyto(self.b_f_rev,self.lstm_ecg_rwts[9])
        np.copyto(self.b_c_rev,self.lstm_ecg_rwts[10])
        np.copyto(self.b_o_rev,self.lstm_ecg_rwts[11])

    def conv_pred(self,test):
        np.copyto(self.conv_ip,test)
        self.conv_reg.input_conv_1 = self.conv_ip.physical_address
        self.conv_reg.k1_1 = self.k1_buff.physical_address
        self.conv_reg.bias1_1 = self.bias1_buff.physical_address
        self.conv_reg.k2_1 = self.k2_buff.physical_address
        self.conv_reg.bias2_1 = self.bias2_buff.physical_address
        self.conv_reg.hidden_curr_1 = self.h1.physical_address
        self.conv_reg.hidden_curr_rev_1 = self.h2.physical_address
        
        self.conv_reg.weights_f_1 = self.w_f_for.physical_address
        self.conv_reg.weights_h_f_1 = self.u_f_for.physical_address
        self.conv_reg.bias_f_1 = self.b_f_for.physical_address

        self.conv_reg.weights_f_rev_1 = self.w_f_rev.physical_address
        self.conv_reg.weights_h_f_rev_1 = self.u_f_rev.physical_address
        self.conv_reg.bias_f_rev_1 = self.b_f_rev.physical_address

        self.conv_reg.weights_icg_1 = self.w_i_for.physical_address
        self.conv_reg.weights_h_icg_1 = self.u_i_for.physical_address
        self.conv_reg.bias_icg_1 = self.b_i_for.physical_address

        self.conv_reg.weights_icg_rev_1 = self.w_i_rev.physical_address
        self.conv_reg.weights_h_icg_rev_1 = self.u_i_rev.physical_address
        self.conv_reg.bias_icg_rev_1 = self.b_i_rev.physical_address

        self.conv_reg.weights_gg_1 = self.w_c_for.physical_address
        self.conv_reg.weights_h_gg_1 = self.u_c_for.physical_address
        self.conv_reg.bias_gg_1 = self.b_c_for.physical_address

        self.conv_reg.weights_gg_rev_1 = self.w_c_rev.physical_address
        self.conv_reg.weights_h_gg_rev_1 = self.u_c_rev.physical_address
        self.conv_reg.bias_gg_rev_1 = self.b_c_rev.physical_address

        self.conv_reg.weights_out_1 = self.w_o_for.physical_address
        self.conv_reg.weights_h_out_1 = self.u_o_for.physical_address
        self.conv_reg.bias_out_1 = self.b_o_for.physical_address

        self.conv_reg.weights_out_rev_1 = self.w_o_rev.physical_address
        self.conv_reg.weights_h_out_rev_1 = self.u_o_rev.physical_address
        self.conv_reg.bias_out_rev_1 = self.b_o_rev.physical_address

    def dense_pred(self,ip1,ip2):
        np.copyto(self.dense_ip_1,ip1)
        np.copyto(self.dense_ip_2,ip2)
        self.dense_reg.input1_1 = self.dense_ip_1.physical_address
        self.dense_reg.input2_1 = self.dense_ip_2.physical_address
        self.dense_reg.output_r_1 = self.dense_op.physical_address
        


In [3]:
bp = bp_predictor()

In [30]:
ppg_h1 = np.zeros(shape=(32,),dtype=np.float32)
ppg_h2 = np.zeros(shape=(32,),dtype=np.float32)
ecg_h1 = np.zeros(shape=(32,),dtype=np.float32)
ecg_h2 = np.zeros(shape=(32,),dtype=np.float32)

In [4]:
ppg_test = np.load('ppg_test.npy')
ecg_test = np.load('ecg_test.npy')

In [5]:
bp.conv_mem()

In [31]:
bp.load_ppg()

In [32]:
bp.conv_pred(ppg_test[0])

In [33]:
bp.conv_cell.write(0x0,0x81)

In [34]:
bp.h2

PynqBuffer([-0.99338126, -0.9906138 ,  0.9961582 ,  0.99395883,
            -0.9971697 , -0.98683286,  0.98923945,  0.9950653 ,
             0.99487644,  0.995038  , -0.9939389 , -0.99673814,
             0.9952721 , -0.98916525,  0.994689  , -0.98330337,
             0.9910501 ,  0.99331737,  0.9931977 ,  0.9974935 ,
            -0.9929964 , -0.99633276,  0.9905854 , -0.9928543 ,
            -0.9922396 ,  0.9872206 , -0.99324363, -0.9940387 ,
            -0.99087054,  0.99536633,  0.9973225 , -0.9898715 ],
           dtype=float32)

In [35]:
np.copyto(ppg_h1,bp.h1)

In [36]:
np.copyto(ppg_h2,bp.h2)

In [37]:
print(ppg_h1,ppg_h2)

[ 0.9978388   0.9968822  -0.99847466  0.99878055 -0.99679023  0.9974456
  0.9966948   0.99715996  0.9978654   0.99742067  0.99674654 -0.99738616
  0.9973108   0.99890184  0.997092    0.9982969   0.9943686   0.9991097
  0.99797565 -0.9982754   0.9963341  -0.9971908   0.9968637  -0.99660647
  0.9957237  -0.99882656 -0.99834645 -0.996566    0.9973355   0.99618894
  0.9977121   0.9967908 ] [-0.99338126 -0.9906138   0.9961582   0.99395883 -0.9971697  -0.98683286
  0.98923945  0.9950653   0.99487644  0.995038   -0.9939389  -0.99673814
  0.9952721  -0.98916525  0.994689   -0.98330337  0.9910501   0.99331737
  0.9931977   0.9974935  -0.9929964  -0.99633276  0.9905854  -0.9928543
 -0.9922396   0.9872206  -0.99324363 -0.9940387  -0.99087054  0.99536633
  0.9973225  -0.9898715 ]


In [38]:
bp.load_ecg()

In [39]:
bp.conv_pred(ecg_test[0])

In [40]:
bp.h1

PynqBuffer([-8.7768650e-01,  9.1769844e-01, -9.4788182e-01,
             8.2079923e-01,  6.9904679e-01,  9.6660167e-01,
             9.5873225e-01,  1.6002855e-01,  8.9895344e-01,
            -9.5328307e-01,  9.6309304e-01,  5.5093634e-01,
             9.7284061e-01,  8.3105206e-01, -9.7856057e-01,
             6.5737921e-01,  7.8073651e-01, -7.2336787e-01,
             8.6948901e-01, -3.8512743e-01, -9.6775353e-01,
            -9.9020582e-01,  8.5587114e-01,  9.2307180e-01,
             9.5761675e-01, -9.8786777e-01, -8.8857299e-01,
            -5.0082675e-04,  9.5770347e-01, -9.5289832e-01,
             9.6141613e-01, -6.2249911e-01], dtype=float32)

In [41]:
bp.h2

PynqBuffer([ 0.9805647 ,  0.98802185, -0.9667315 , -0.9842517 ,
             0.9393509 ,  0.9829229 , -0.0552928 ,  0.91280276,
             0.97893393, -0.68718517, -0.95842814,  0.9640458 ,
            -0.95319974, -0.9876452 ,  0.9856134 ,  0.9830661 ,
            -0.97358024,  0.9614921 ,  0.97891194, -0.98419476,
             0.9768409 , -0.97897094, -0.976111  , -0.9692732 ,
             0.9738459 , -0.98512405,  0.9495432 ,  0.9744542 ,
            -0.9700135 , -0.9738369 , -0.9216797 ,  0.96916956],
           dtype=float32)

In [42]:
np.copyto(ecg_h1,bp.h1)
np.copyto(ecg_h2,bp.h2)

In [51]:
bp.dense_mem()

In [24]:
type(ecg_h1)

pynq.buffer.PynqBuffer

In [46]:
ip1 = np.concatenate((ppg_h1,ppg_h2))
ip2 = np.concatenate((ecg_h1,ecg_h2))

In [52]:
bp.dense_pred(ip1,ip2)

In [53]:
bp.dense_op

PynqBuffer([0., 0.], dtype=float32)

In [54]:
bp.dense_reg

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  input1_1 = Register(input1=write-only),
  input1_2 = Register(input1=write-only),
  input2_1 = Register(input2=write-only),
  input2_2 = Register(input2=write-only),
  output_r_1 = Register(output_r=write-only),
  output_r_2 = Register(output_r=write-only)
}

In [57]:
bp.dense_cell.write(0x0,0x81)

In [58]:
bp.dense_pred(ip1,ip2)

In [59]:
bp.dense_op

PynqBuffer([122.25319,  55.97493], dtype=float32)